In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests 
import json
import pandas_profiling
from numpy import array

import quandl
quandl_api = 'YDy5xD85FUyhACHzsunX'
quandl.ApiConfig.api_key = 'YDy5xD85FUyhACHzsunX'


import eia
api = eia.API('aeb9ba252acba96a087cdd98626d5fc2')


# Importing Data from Quandl and Energy Information Administration #

##### Quandl: ### 

<a href="https://www.quandl.com/data/UICT-United-Nations-Information-and-Communication-Technology?keyword=internet%20use" target="_blank">*Percent of Population Connected to Internet*</a>


##### Energy Information Administration: ###

<a href="https://www.eia.gov/international/data/world/electricity/electricity-consumption?pd=2&p=0000002&u=0&f=A&v=mapbubble&a=-&i=none&vo=value&&t=C&g=00000000000000000000000000000000000000000000000001&l=249-ruvvvvvfvtvnvv1vrvvvvfvvvvvvfvvvou20evvvvvvvvvvnvvvs0008&s=315532800000&e=1546300800000" target="_blank">*Total Electricity Consumption*</a>


<a href="https://www.eia.gov/international/data/world/total-energy/total-energy-consumption?pd=44&p=0000000010000000000000000000000000000000000000000000000000u06&u=0&f=A&v=mapbubble&a=-&i=none&vo=value&&t=C&g=00000000000000000000000000000000000000000000000001&l=249-ruvvvvvfvtvnvv1vrvvvvfvvvvvvfvvvou20evvvvvvvvvvnvvvs0008&s=315532800000&e=1514764800000" target="_blank">*Total Energy Consumption*</a>


<a href="https://www.eia.gov/international/data/world/other-statistics/population-by-country?pd=4702&p=000000000000000000000000000000000000000000000000000000001&u=0&f=A&v=mapbubble&a=-&i=none&vo=value&t=C&g=00000000000000000000000000000000000000000000000001&l=249-ruvvvvvfvtvnvv1vrvvvvfvvvvvvfvvvou20evvvvvvvvvvnvvvs0008&s=315532800000&e=1546300800000&" target="_blank">*Total Population*</a>



## Quandl Internet Usage Data Import  #

In [42]:
#Manually entered country codes from Quandl documentation as a list. 

country_codes = ['AFG', 'ALB', 'DZA', 'AND', 'AGO', 'ATG', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLZ', 'BEN', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 
'BRN', 'BGR', 'BFA', 'BDI', 'KHM', 'CMR', 'CAN', 'CPV', 'CAF', 'TCD', 'CHL', 'CHN', 'COL', 'COM', 'COG', 'COK', 'CRI', 'CIV', 'HRV', 'CUB', 'CYP', 'CZE', 'PRK', 'DNK', 
'COD', 'DJI', 'DMA', 'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'ETH', 'FJI', 'FIN', 'FRA', 'GAB', 'GMB', 'GEO', 'DEU', 'GHA', 'GRC', 'GRD', 'GTM', 'GIN', 'GNB',
'GUY', 'HTI', 'HND', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KIR', 'KWT', 'KGZ', 'LAO', 'LVA', 'LBN',
'LSO', 'LBR', 'LBY', 'LTU', 'LUX', 'MDG', 'MYS', 'MDV', 'MLT', 'MHL', 'MWI', 'MRT', 'MLI', 'MUS', 'MEX', 'FSM', 'MCO', 'MNG', 'MNE', 'MAR', 'MOZ', 'MMR', 'NAM', 'NRU',
'NPL', 'NLD', 'NZL', 'NIC', 'NER', 'NGA', 'NIU', 'NOR', 'OMN', 'PAK', 'PLW', 'PAN', 'PNG', 'PRY', 'PER', 'PHL', 'POL', 'PRT', 'QAT', 'KOR', 'MDA', 'ROU', 'RUS', 'RWA',
'KNA', 'LCA', 'VCT', 'WSM', 'SMR', 'STP', 'SAU', 'SEN', 'SRB', 'SYC', 'SLE', 'SGP', 'SVK', 'SVN', 'SLB', 'SOM', 'ZAF', 'SSD', 'ESP', 'LKA', 'SDN', 'SUR', 'SWZ', 'SWE', 
'CHE', 'SYR', 'TJK', 'THA', 'MKD', 'TLS', 'TGO', 'TON', 'TTO', 'TUN', 'TUR', 'TKM', 'TUV', 'UGA', 'UKR', 'ARE', 'GBR', 'TZA', 'USA', 'URY', 'UZB', 'VUT', 'VEN', 'VNM',
'YEM', 'ZMB', 'ZWE']


#Created a for loop to append country codes with the Quandl database code 'UICT/INT_'.
appended_country_codes = []

for country in country_codes:
    appended_country_codes.append('UICT/INT_' + str(country))
    
    
#Imported data for each country into one dataframe.
internet_df = quandl.get(appended_country_codes, authtoken= "YDy5xD85FUyhACHzsunX")


#Changed column names to country codes.
internet_df.columns = country_codes


#Dropped rows so that dataframe starts at 1990.
internet_df = internet_df.drop(internet_df.index[0:18])


#Set internet_df index as the year.
internet_df = internet_df.set_index(pd.DatetimeIndex(internet_df.index).year)

In [43]:
internet_df.head(25)

,AFG,ALB,DZA,AND,AGO,ATG,ARG,ARM,AUS,AUT,...,TZA,USA,URY,UZB,VUT,VEN,VNM,YEM,ZMB,ZWE
Date,,,,,,,,,,,,,,,,,,,,,
1990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.585095,0.130369,...,0.000000,0.784729,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.097204,0.259082,...,NaN,1.163194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,NaN,NaN,NaN,NaN,NaN,NaN,0.002993,NaN,1.768765,0.642818,...,NaN,1.724203,NaN,NaN,NaN,0.012082,NaN,NaN,NaN,NaN
1993,NaN,NaN,NaN,NaN,NaN,NaN,0.029527,NaN,1.974611,0.765343,...,NaN,2.271673,NaN,NaN,NaN,0.041587,NaN,NaN,NaN,NaN
1994,NaN,NaN,0.000361,NaN,NaN,NaN,0.043706,0.009117,2.232101,1.393423,...,NaN,4.862781,0.062474,NaN,NaN,0.055487,NaN,NaN,0.006776,0.001739
1995,NaN,0.011169,0.001769,NaN,NaN,2.200769,0.086277,0.052743,2.759655,1.890211,...,NaN,9.237088,0.310129,0.001527,NaN,0.122215,NaN,NaN,0.008784,0.007684
1996,NaN,0.032197,0.001739,1.526601,0.000776,2.858450,0.141955,0.094573,3.275250,6.909162,...,0.001622,16.419353,1.847472,0.004285,0.056846,0.248261,0.000135,0.000621,0.009069,0.016790
1997,NaN,0.048594,0.010268,3.050175,0.005674,3.480537,0.280340,0.111651,16.369358,9.533974,...,0.007900,21.616401,3.363240,0.010537,0.139506,0.390959,0.003982,0.015025,0.009330,0.033080
1998,NaN,0.065027,0.020239,6.886209,0.018454,4.071716,0.830767,0.128659,30.813239,15.421219,...,0.009244,30.093197,6.987069,0.020754,0.274186,1.383990,0.013079,0.023323,0.030226,0.081648


## Energy Information Administration Data Import

### Electricity Consumption


In [44]:
elec_country_codes = ['AFG','ALB','DZA','ASM','AGO','ATA','ATG','ARG','ARM','ABW','AUS','AUT','AZE','BHR',
 'BGD','BRB','BLR','BEL','BLZ','BEN','BMU','BTN','BOL','BIH','BWA','BRA','VGB','BRN','BGR','BFA','MMR','BDI','CPV',
 'KHM','CMR','CAN','CYM','CAF','TCD','CHL','CHN','COL','COM','COG','COD','COK','CRI','HRV','CUB','CYP','CZE','CIV',
 'DNK','DJI','DMA','DOM','ECU','EGY','SLV','GNQ','ERI','EST','SWZ','ETH','FLK','FRO','FJI','FIN','CSK','SCG','SUN',
 'YUG','FRA','GUF','PYF','GAB','GMB','GEO','DEU','DDR','DEU','GHA','GIB','GRC','GRL','GRD','GLP','GUM','GTM','GIN',
 'GNB','GUY','HTI','HIT','HND','HKG','HUN','ISL','IND','IDN','IRN','IRQ','IRL','ISR','ITA','JAM','JPN','JOR','KAZ',
 'KEN','KIR','XKS', 'KWT','KGZ','LAO','LVA','LBN','LSO','LBR','LBY','LTU','LUX','MAC','MDG','MWI','MYS''MDV','MLI',
 'MLT','MTQ','MRT','MUS','MEX','MDA','MNG','MNE','MSR','MAR','MOZ','NAM','NRU''NPL','NLD','NLD','NCL','NZL','NIC',
 'NER','NGA','NIU','PRK','MKD','NOR','OMN','PAK','PSE','PAN','PNG','PRY','PER','PHL','POL','PRT','PRI','QAT','REU',
 'ROU','RUS','RWA','SHN','KNA','LCA','SPM','VCT','WSM','STP','SAU','SEN','SRB','SYC','SLE','SGP','SVK','SVN','SLB',
 'SOM','ZAF','KOR','SSD','ESP','LKA','SDN','SUR','SWE','CHE','SYR','TWN','TJK','TZA','THA','BHS','TLS','TGO','TON',
 'TTO','TUN','TUR','TKM','TCA','USI','VIR','UGA','UKR','ARE','GBR','USA','URY','UZB','VUT','VEN','VNM','WAK','ESH',
 'YEM','ZMB','ZWE','ZMB','ZWE']

In [45]:
appended_elec_country_codes = []

for country in elec_country_codes:
    appended_elec_country_codes.append('INTL.2-2-' + str(country) + '-BKWH.A')

In [70]:
appended_elec_country_codes

['INTL.2-2-AFG-BKWH.A',
 'INTL.2-2-ALB-BKWH.A',
 'INTL.2-2-DZA-BKWH.A',
 'INTL.2-2-ASM-BKWH.A',
 'INTL.2-2-AGO-BKWH.A',
 'INTL.2-2-ATA-BKWH.A',
 'INTL.2-2-ATG-BKWH.A',
 'INTL.2-2-ARG-BKWH.A',
 'INTL.2-2-ARM-BKWH.A',
 'INTL.2-2-ABW-BKWH.A',
 'INTL.2-2-AUS-BKWH.A',
 'INTL.2-2-AUT-BKWH.A',
 'INTL.2-2-AZE-BKWH.A',
 'INTL.2-2-BHR-BKWH.A',
 'INTL.2-2-BGD-BKWH.A',
 'INTL.2-2-BRB-BKWH.A',
 'INTL.2-2-BLR-BKWH.A',
 'INTL.2-2-BEL-BKWH.A',
 'INTL.2-2-BLZ-BKWH.A',
 'INTL.2-2-BEN-BKWH.A',
 'INTL.2-2-BMU-BKWH.A',
 'INTL.2-2-BTN-BKWH.A',
 'INTL.2-2-BOL-BKWH.A',
 'INTL.2-2-BIH-BKWH.A',
 'INTL.2-2-BWA-BKWH.A',
 'INTL.2-2-BRA-BKWH.A',
 'INTL.2-2-VGB-BKWH.A',
 'INTL.2-2-BRN-BKWH.A',
 'INTL.2-2-BGR-BKWH.A',
 'INTL.2-2-BFA-BKWH.A',
 'INTL.2-2-MMR-BKWH.A',
 'INTL.2-2-BDI-BKWH.A',
 'INTL.2-2-CPV-BKWH.A',
 'INTL.2-2-KHM-BKWH.A',
 'INTL.2-2-CMR-BKWH.A',
 'INTL.2-2-CAN-BKWH.A',
 'INTL.2-2-CYM-BKWH.A',
 'INTL.2-2-CAF-BKWH.A',
 'INTL.2-2-TCD-BKWH.A',
 'INTL.2-2-CHL-BKWH.A',
 'INTL.2-2-CHN-BKWH.A',
 'INTL.2-2-COL-B

In [97]:
hong_kong = pd.DataFrame(api.data_by_series(series='INTL.2-2-HKG-BKWH.A'))
united_states = pd.DataFrame(api.data_by_series(series='INTL.2-2-USA-BKWH.A'))
test = pd.concat([united_states, hong_kong], axis = 1)

In [98]:
'''
This is what I want the dataframe to look like, with all of the countries.
'''

test.head(50)

,"Electricity net consumption, United States, Annual (billion kilowatthours)","Electricity net consumption, Hong Kong, Annual (billion kilowatthours)"
1980,2094.449000,10.130000
1981,2147.102862,10.724000
1982,2086.441352,11.526000
1983,2150.954584,13.055000
1984,2285.796394,13.971000
1985,2323.973852,14.774000
1986,2368.753052,16.382000
1987,2457.272219,20.033000
1988,2578.062495,21.387000
1989,2755.635153,20.746000


In [107]:
test_df = pd.DataFrame(api.data_by_series(series='INTL.2-2-AFG-BKWH.A'))
df_list = pd.DataFrame()

for i in appended_elec_country_codes:
    df_list = df_list.append(pd.DataFrame(api.data_by_series(series= i)))
    
electricity_df = pd.DataFrame(pd.concat([df_list], axis = 1))
    

KeyError: 'series'

In [108]:
electricity_df

[        Electricity net consumption, Afghanistan, Annual (billion kilowatthours)
 1980                                             0.876060                       
 1981                                             0.922560                       
 1982                                             0.885360                       
 1983                                             0.930000                       
 1984                                             0.947670                       
 1985                                             0.961620                       
 1986                                             1.055550                       
 1987                                             1.132740                       
 1988                                             1.004400                       
 1989                                             1.012770                       
 1990                                             1.021140                       
 1991           

In [55]:
'''
Found this online to maybe help

(https://medium.com/analytics-vidhya/use-python-to-pull-energy-data-from-the-us-department-of-energys-api-11d6f724927e)
'''


electricity_df = []

for i in range(len(appended_elec_country_codes)):
    
    url = 'http://api.eia.gov/series/?api_key=aeb9ba252acba96a087cdd98626d5fc2&series_id=' + appended_elec_country_codes[i]
    
    r = requests.get(url)
    json_data = r.json()
    
    if r.status_code == 200:
        print('Success!' + appended_elec_country_codes[i])
    else:
        print('Error')
    
    df = pd.DataFrame(json_data.get('series')[0].get('data'), columns = ['Data', appended_elec_country_codes[i]])
    
    
    electricity_df.append(df)



Success!INTL.2-2-AFG-BKWH.A
Success!INTL.2-2-ALB-BKWH.A
Success!INTL.2-2-DZA-BKWH.A
Success!INTL.2-2-ASM-BKWH.A
Success!INTL.2-2-AGO-BKWH.A
Success!INTL.2-2-ATA-BKWH.A
Success!INTL.2-2-ATG-BKWH.A
Success!INTL.2-2-ARG-BKWH.A
Success!INTL.2-2-ARM-BKWH.A
Success!INTL.2-2-ABW-BKWH.A
Success!INTL.2-2-AUS-BKWH.A
Success!INTL.2-2-AUT-BKWH.A
Success!INTL.2-2-AZE-BKWH.A
Success!INTL.2-2-BHR-BKWH.A
Success!INTL.2-2-BGD-BKWH.A
Success!INTL.2-2-BRB-BKWH.A
Success!INTL.2-2-BLR-BKWH.A
Success!INTL.2-2-BEL-BKWH.A
Success!INTL.2-2-BLZ-BKWH.A
Success!INTL.2-2-BEN-BKWH.A
Success!INTL.2-2-BMU-BKWH.A
Success!INTL.2-2-BTN-BKWH.A
Success!INTL.2-2-BOL-BKWH.A
Success!INTL.2-2-BIH-BKWH.A
Success!INTL.2-2-BWA-BKWH.A
Success!INTL.2-2-BRA-BKWH.A
Success!INTL.2-2-VGB-BKWH.A
Success!INTL.2-2-BRN-BKWH.A
Success!INTL.2-2-BGR-BKWH.A
Success!INTL.2-2-BFA-BKWH.A
Success!INTL.2-2-MMR-BKWH.A
Success!INTL.2-2-BDI-BKWH.A
Success!INTL.2-2-CPV-BKWH.A
Success!INTL.2-2-KHM-BKWH.A
Success!INTL.2-2-CMR-BKWH.A
Success!INTL.2-2-CAN

TypeError: 'NoneType' object is not subscriptable

In [5]:
'''Found this code online, trying to figure out how to use it with a for loop to append a dataframe
with columns for each electricity_country_code.


(https://techrando.com/2019/06/26/how-to-use-the-eia-api-to-pull-live-data-directly-into-python/)

'''



def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    df = pd.DataFrame(series_search)
    return df

def main():
    """
    Run main script
    """
    #Create EIA API using your specific API key
    api_key = "aeb9ba252acba96a087cdd98626d5fc2"
    api = eia.API(api_key)
    #Declare desired series ID
    series_ID= 'INTL.2-2-BGR-BKWH.A'
    df=retrieve_time_series(api, series_ID)
    #Print the returned dataframe df
    print(df)

